In [1]:
# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
import datetime
from scipy import stats, integrate

In [2]:
herepath = pd.read_csv('data/MOW_Path_Speed_5Min_2014_DOW_SpeedLimit.csv', encoding='utf-8')
herepath.head()

,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,SPDLIMIT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT_5,PCT_50,PCT_85,DOW
0,998969850F,2014-01-01 13:55,167,186,72,61.0,0.0,61,61,10,61,61,61,Wednesday
1,998969850F,2014-01-02 11:05,133,186,72,60.0,0.0,60,60,10,60,60,60,Thursday
2,998969850F,2014-01-02 13:20,160,186,72,61.0,0.0,61,61,10,61,61,61,Thursday
3,998969850F,2014-01-02 14:30,174,186,72,52.0,0.0,52,52,10,52,52,52,Thursday
4,998969850F,2014-01-03 09:05,109,186,72,57.0,0.0,57,57,10,57,57,57,Friday


In [3]:
#For now, enter the weekdays desire here in full
Weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Weekday

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [4]:
Weekend = ['Saturday','Sunday']
Weekend

['Saturday', 'Sunday']

In [5]:
Both = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Both

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [6]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe(w,x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        link = v[v['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        link = x[x['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [26]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe2(x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [8]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsBase(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y = pd.DataFrame(columns=[])
    
    y['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thAll'] = epoct
    y['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
   
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
      

    
    y = np.round(y,3)
    return z

In [9]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsALL(x):
    y = pd.DataFrame(columns=[])
    
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thAll'] = epoct
    y['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
    y = np.round(y,3)
    return y

In [10]:
def IndecesALLAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_AM'] = y['SPDLIMIT'].div(y['MEAN'])
    z['PTI_SL_AM'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85thALL_AM'] = y['PcntSpeed_85thAll'].div(y['MEAN'])
    z['PTI_85thALL_AM'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_AM'] = (y['MEAN'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [11]:
def IndecesALLMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_MD'] = y['SPDLIMIT'].div(y['MEAN'])
    z['PTI_SL_MD'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85th_MD'] = y['PcntSpeed_85thAll'].div(y['MEAN'])
    z['PTI_85th_MD'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_MD'] = (y['MEAN'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [12]:
def IndecesALLPM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_PM'] = y['SPDLIMIT'].div(y['MEAN'])
    z['PTI_SL_PM'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85th_PM'] = y['PcntSpeed_85thAll'].div(y['MEAN'])
    z['PTI_85th_PM'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_PM'] = (y['MEAN'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [13]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWd(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)

    
    y['PcntSpeed_85thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWd'] = epoct
    y['PcntInterval_85thWd'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWd'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [14]:
def IndecesWdAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_AM'] = y['PcntSpeed_85thWd'].div(y['MEAN'])
    z['PTI_85thWd_AM'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [15]:
def IndecesWdMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_MD'] = y['PcntSpeed_85thWd'].div(y['MEAN'])
    z['PTI_85thWd_MD'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [16]:
def IndecesWdPM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_PM'] = y['PcntSpeed_85thWd'].div(y['MEAN'])
    z['PTI_85thWd_PM'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [17]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWend(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thWend'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWend'] = epoct
    y['PcntInterval_85thWend'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWend'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [18]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdtime(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_60thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_85thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_Wdtime'] = epoct
    y['PcntInterval_Wdtime'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_Wdtime'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [19]:
def IndecesWDtimeAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_AM'] = y['PcntSpeed_60thWdtime'].div(y['MEAN'])
    z['PTI_60thWdtime_AM'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_AM'] = y['PcntSpeed_85thWdtime'].div(y['MEAN'])
    z['PTI_85thWdtime_AM'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [20]:
def IndecesWDtimeMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_MD'] = y['PcntSpeed_60thWdtime'].div(y['MEAN'])
    z['PTI_60thWdtime_MD'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_MD'] = y['PcntSpeed_85thWdtime'].div(y['MEAN'])
    z['PTI_85thWdtime_MD'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [21]:
def IndecesWDtimePM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_PM'] = y['PcntSpeed_60thWdtime'].div(y['MEAN'])
    z['PTI_60thWdtime_PM'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_PM'] = y['PcntSpeed_85thWdtime'].div(y['MEAN'])
    z['PTI_85thWdtime_PM'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [22]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayAM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayAM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayAM'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayAM'] = epoct
    y['PcntInterval_WdayAM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayAM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [23]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayMD(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayMD'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayMD'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayMD'] = epoct
    y['PcntInterval_WdayMD'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayMD'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [24]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayPM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayPM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayPM'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayPM'] = epoct
    y['PcntInterval_WdayPM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayPM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [27]:
#For all days and all time periods
All = timeframe2(herepath,0,288,False,Both)
bylink = HEREstatsALL(All)
bylink = bylink.drop(['MEAN','SPDLIMIT'], axis = 1)
All = pd.DataFrame()
bylink.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll
LINK_DIR,,,,,
107030432T,48.461,19.260,5746,5.466,10.444
107030439T,44.734,17.396,7857,7.474,9.929
107030440T,45.355,17.086,7325,6.968,10.303
107030444F,47.219,16.154,9311,8.857,11.209
107030445F,47.840,16.775,9256,8.805,11.250


In [29]:
#for weekdays all links
Wkd = timeframe2(herepath,0,288,True,Weekday)
SpeedWd = HEREstatsWd(Wkd)
SpeedWd = SpeedWd.drop(['MEAN','SPDLIMIT'], axis = 1)
Wkd = pd.DataFrame()
SpeedWd.head()

,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd
LINK_DIR,,,,,
107030432T,47.840,19.260,5023,6.682,10.344
107030439T,44.734,17.396,6775,9.013,9.937
107030440T,45.355,17.396,6339,8.433,10.262
107030444F,47.219,15.532,8045,10.703,11.301
107030445F,47.840,16.775,8008,10.653,11.282


In [30]:
#for weekends all links
Wend = timeframe2(herepath,0,288,True,Weekend)
SpeedWend = HEREstatsWend(Wend)
SpeedWend = SpeedWend.drop(['MEAN','SPDLIMIT'], axis = 1)
Wend = pd.DataFrame()
SpeedWend.head()

,PcntSpeed_85thWend,TotalIntervals_85thWend,PcntInterval_85thWend,StdDev_85thWend
LINK_DIR,,,,
107030432T,50.325,723,2.414,10.988
107030439T,45.355,1082,3.612,9.874
107030440T,45.355,986,3.292,10.532
107030444F,47.840,1266,4.227,10.461
107030445F,49.083,1248,4.167,10.760


In [31]:
#Weekday Between 6AM and 8PM all links
Wdtime = timeframe2(herepath,72,240,True,Weekday)
SpeedWdtime = HEREstatsWdtime(Wdtime)
SpeedWdtime = SpeedWdtime.drop(['MEAN','SPDLIMIT'], axis = 1)
Wdtime = pd.DataFrame()
SpeedWdtime.head()

,PcntSpeed_60thWdtime,PcntSpeed_85thWdtime,PcntSpeed_5thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime
LINK_DIR,,,,,,
107030432T,41.627,47.840,18.639,4458,5.931,10.412
107030439T,38.521,44.734,17.396,6233,8.292,9.992
107030440T,37.899,45.355,16.775,5891,7.837,10.269
107030444F,40.198,47.219,14.911,7538,10.028,11.354
107030445F,40.384,47.219,16.154,7355,9.785,11.290


In [32]:
#Stats between 6AM and 9AM all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
SpeedWdayAM = HEREstatsWdayAM(WdtimeAM)
SpeedWdayAM = SpeedWdayAM.drop(['MEAN','SPDLIMIT'], axis = 1)
WdtimeAM = pd.DataFrame()
SpeedWdayAM.head()

,PcntSpeed_5thWdayAM,AvgSpeed_WdayAM,TotalIntervals_WdayAM,PcntInterval_WdayAM,StdDev_WdayAM
LINK_DIR,,,,,
107030432T,19.260,38.661,735,0.978,10.649
107030439T,16.775,34.048,991,1.318,10.676
107030440T,16.154,34.625,852,1.133,11.012
107030444F,14.911,33.387,1313,1.747,11.931
107030445F,15.532,33.241,1340,1.783,11.442


In [34]:
#Stats between 9AM and 3PM all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
SpeedWdayMD = HEREstatsWdayMD(WdtimeMD)
SpeedWdayMD = SpeedWdayMD.drop(['MEAN','SPDLIMIT'], axis = 1)
WdtimeMD = pd.DataFrame()
SpeedWdayMD.head()

,PcntSpeed_5thWdayMD,AvgSpeed_WdayMD,TotalIntervals_WdayMD,PcntInterval_WdayMD,StdDev_WdayMD
LINK_DIR,,,,,
107030432T,18.639,38.016,2168,2.884,10.330
107030439T,18.639,35.201,3283,4.368,9.513
107030440T,17.396,34.843,3159,4.203,9.932
107030444F,16.775,37.005,3910,5.202,10.765
107030445F,17.396,37.109,3840,5.109,10.932


In [ ]:
#Stats between 3PM and 6PM all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
SpeedWdayPM = HEREstatsWdayPM(WdtimePM)
SpeedWdayPM = SpeedWdayPM.drop(['MEAN','SPDLIMIT'], axis = 1)
WdtimePM = pd.DataFrame()
SpeedWdayPM.head()

In [142]:
#Calculates AM Indeces for all 365 days for all links
AllAM = timeframe2(herepath,72,108,False,Both)
HstatsAM = HEREstatsALL(AllAM)
IndecesAllAM = IndecesALLAM(HstatsAM)
IndecesAllAM.head()

In [144]:
#Calculates MD Indeces for all 365 days for all links
AllMD = timeframe2(herepath,108,180,False,Both)
HstatsMD = HEREstatsALL(AllMD)
IndecesAllMD = IndecesALLMD(HstatsMD)
IndecesAllMD.head()

In [146]:
#Calculates PM Indeces for all 365 days for all links
AllPM = timeframe2(herepath,180,216,False,Both)
HstatsPM = HEREstatsALL(AllPM)
IndecesAllPM = IndecesALLPM(HstatsPM)
IndecesAllPM.head()

In [148]:
#Calculates weekday non-holiday indeces for AM for all links
WdAM = timeframe2(herepath,72,108,True,Weekday)
HstatsWdAM = HEREstatsWd(WdAM)
IndecesWdayAM = IndecesWdAM(HstatsWdAM)
IndecesWdayAM.head()

In [150]:
#Calculates weekday non-holiday indeces for MD for all links
WdMD = timeframe2(herepath,108,180,True,Weekday)
HstatsWdMD = HEREstatsWd(WdMD)
IndecesWdayMD = IndecesWdMD(HstatsWdMD)
IndecesWdayMD.head()

In [152]:
#Calculates weekday non-holiday indeces for PM for all links
WdPM = timeframe2(herepath,180,216,True,Weekday)
HstatsWdPM = HEREstatsWd(WdPM)
IndecesWdayPM = IndecesWdPM(HstatsWdPM)
IndecesWdayPM.head()

In [154]:
#Calculates daytime indeces for AM for all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
HstatsWdtimeAM = HEREstatsWdtime(WdtimeAM)
IndecesWdtimeAM = IndecesWdtimeAM(HstatsWdtimeAM)
IndecesWdtimeAM.head()

In [156]:
#Calculates daytime indeces for MD for all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
HstatsWdtimeMD = HEREstatsWdtime(WdtimeMD)
IndecesWdtimeMD = IndecesWdtimeMD(HstatsWdtimeMD)
IndecesWdtimeMD.head()

In [158]:
#Calculates daytime indeces for PM for all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
HstatsWdtimePM = HEREstatsWdtime(WdtimePM)
IndecesWdtimePM = IndecesWdtimePM(HstatsWdtimePM)
IndecesWdtimePM.head()

In [31]:
master = bylink

In [32]:
master = pd.merge(master, SpeedWd, left_index = True, right_index = True)
SpeedWd = pd.DataFrame(columns = [])
master.head()

,MEAN_x,SPDLIMIT_x,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_y,SPDLIMIT_y,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd
LINK_DIR,,,,,,,,,,,,,,
107030432T,38.553,49.704,48.461,19.260,5746,5.466,10.444,38.326,49.704,47.840,19.260,5023,6.682,10.344
107030439T,35.148,44.734,44.734,17.396,7857,7.474,9.929,35.082,44.734,44.734,17.396,6775,9.013,9.937
107030440T,34.863,44.734,45.355,17.086,7325,6.968,10.303,34.991,44.734,45.355,17.396,6339,8.433,10.262
107030444F,36.180,44.734,47.219,16.154,9311,8.857,11.209,35.919,44.734,47.219,15.532,8045,10.703,11.301
107030445F,36.602,44.734,47.840,16.775,9256,8.805,11.250,36.240,44.734,47.840,16.775,8008,10.653,11.282
107030447F,25.206,44.734,39.142,4.970,8823,8.393,12.731,24.723,44.734,38.521,4.349,7693,10.234,12.697
107030448F,32.218,44.734,47.219,7.456,9083,8.641,13.905,31.948,44.734,47.219,7.456,7854,10.449,13.949
107030449F,36.827,44.734,48.461,16.154,8846,8.415,11.806,36.328,44.734,48.461,15.532,7641,10.165,11.948
107030455F,28.776,44.734,41.006,9.941,13325,12.676,11.403,28.856,44.734,41.006,9.941,11541,15.354,11.401


In [33]:
master = pd.merge(master, SpeedWend, left_index = True, right_index = True)
SpeedWend = pd.DataFrame(columns = [])
master.head()

,MEAN_x,SPDLIMIT_x,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_y,SPDLIMIT_y,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,MEAN,SPDLIMIT,PcntSpeed_85thWend,TotalIntervals_85thWend,PcntInterval_85thWend,StdDev_85thWend
LINK_DIR,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,49.704,48.461,19.260,5746,5.466,10.444,38.326,49.704,47.840,19.260,5023,6.682,10.344,40.127,49.704,50.325,723,2.414,10.988
107030439T,35.148,44.734,44.734,17.396,7857,7.474,9.929,35.082,44.734,44.734,17.396,6775,9.013,9.937,35.565,44.734,45.355,1082,3.612,9.874
107030440T,34.863,44.734,45.355,17.086,7325,6.968,10.303,34.991,44.734,45.355,17.396,6339,8.433,10.262,34.040,44.734,45.355,986,3.292,10.532
107030444F,36.180,44.734,47.219,16.154,9311,8.857,11.209,35.919,44.734,47.219,15.532,8045,10.703,11.301,37.839,44.734,47.840,1266,4.227,10.461
107030445F,36.602,44.734,47.840,16.775,9256,8.805,11.250,36.240,44.734,47.840,16.775,8008,10.653,11.282,38.929,44.734,49.083,1248,4.167,10.760
107030447F,25.206,44.734,39.142,4.970,8823,8.393,12.731,24.723,44.734,38.521,4.349,7693,10.234,12.697,28.494,44.734,42.248,1130,3.773,12.478
107030448F,32.218,44.734,47.219,7.456,9083,8.641,13.905,31.948,44.734,47.219,7.456,7854,10.449,13.949,33.943,44.734,47.840,1229,4.103,13.500
107030449F,36.827,44.734,48.461,16.154,8846,8.415,11.806,36.328,44.734,48.461,15.532,7641,10.165,11.948,39.994,44.734,50.325,1205,4.023,10.318
107030455F,28.776,44.734,41.006,9.941,13325,12.676,11.403,28.856,44.734,41.006,9.941,11541,15.354,11.401,28.258,44.734,40.074,1784,5.956,11.400


In [162]:
master = pd.merge(master, SpeedWdtime, left_index = True, right_index = True)
SpeedWdtime = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,PcntSpeed_85thWend,TotalIntervals_85thWend,PcntInterval_85thWend,StdDev_85thWend,PcntSpeed_60thWdtime,PcntSpeed_85thWdtime,PcntSpeed_5thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime
LINK_DIR,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,46.598,330,1.157,10.751,36.035,41.627,18.018,2587,3.536,8.938


In [163]:
master = pd.merge(master, SpeedWdayAM, left_index = True, right_index = True)
SpeedWdayAM = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,PcntSpeed_85thWdtime,PcntSpeed_5thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime,PcntSpeed_5thWdayAM,AvgSpeed_WdayAM,TotalIntervals_WdayAM,PcntInterval_WdayAM,StdDev_WdayAM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,41.627,18.018,2587,3.536,8.938,18.018,33.838,440,0.728,8.698


In [164]:
master = pd.merge(master, SpeedWdayMD, left_index = True, right_index = True)
SpeedWdayMD = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,PcntSpeed_5thWdayAM,AvgSpeed_WdayAM,TotalIntervals_WdayAM,PcntInterval_WdayAM,StdDev_WdayAM,PcntSpeed_5thWdayMD,AvgSpeed_WdayMD,TotalIntervals_WdayMD,PcntInterval_WdayMD,StdDev_WdayMD
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,18.018,33.838,440,0.728,8.698,18.639,34.014,1367,1.899,8.661


In [165]:
master = pd.merge(master, SpeedWdayPM, left_index = True, right_index = True)
SpeedWdayPM = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,PcntSpeed_5thWdayMD,AvgSpeed_WdayMD,TotalIntervals_WdayMD,PcntInterval_WdayMD,StdDev_WdayMD,PcntSpeed_5thWdayPM,AvgSpeed_WdayPM,TotalIntervals_WdayPM,PcntInterval_WdayPM,StdDev_WdayPM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,18.639,34.014,1367,1.899,8.661,16.775,31.642,651,0.962,9.324


In [166]:
master = pd.merge(master, IndecesAllAM, left_index = True, right_index = True)
IndecesAllAM = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,PcntSpeed_5thWdayPM,AvgSpeed_WdayPM,TotalIntervals_WdayPM,PcntInterval_WdayPM,StdDev_WdayPM,TTI_SL_AM,PTI_SL_AM,TTI_85thALL_AM,PTI_85thALL_AM,BTI_AM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,16.775,31.642,651,0.962,9.324,1.309,2.4,1.255,2.3,0.833


In [167]:
master = pd.merge(master, IndecesAllMD, left_index = True, right_index = True)
IndecesAllMD = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,TTI_SL_AM,PTI_SL_AM,TTI_85thALL_AM,PTI_85thALL_AM,BTI_AM,TTI_SL_MD,PTI_SL_MD,TTI_85th_MD,PTI_85th_MD,BTI_MD
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,1.309,2.4,1.255,2.3,0.833,1.311,2.483,1.238,2.345,0.894


In [168]:
master = pd.merge(master, IndecesAllPM, left_index = True, right_index = True)
IndecesAllPM = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,TTI_SL_MD,PTI_SL_MD,TTI_85th_MD,PTI_85th_MD,BTI_MD,TTI_SL_PM,PTI_SL_PM,TTI_85th_PM,PTI_85th_PM,BTI_PM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,1.311,2.483,1.238,2.345,0.894,1.393,2.769,1.277,2.538,0.988


In [169]:
master = pd.merge(master, IndecesWdayAM, left_index = True, right_index = True)
IndecesWdayAM = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,TTI_85th_MD,PTI_85th_MD,BTI_MD,TTI_SL_PM,PTI_SL_PM,TTI_85th_PM,PTI_85th_PM,BTI_PM,TTI_85thWd_AM,PTI_85thWd_AM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,1.238,2.345,0.894,1.393,2.769,1.277,2.538,0.988,1.249,2.345


In [170]:
master = pd.merge(master, IndecesWdayMD, left_index = True, right_index = True)
IndecesWdayMD = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,BTI_MD,TTI_SL_PM,PTI_SL_PM,TTI_85th_PM,PTI_85th_PM,BTI_PM,TTI_85thWd_AM,PTI_85thWd_AM,TTI_85thWd_MD,PTI_85thWd_MD
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,0.894,1.393,2.769,1.277,2.538,0.988,1.249,2.345,1.242,2.267


In [171]:
master = pd.merge(master, IndecesWdayPM, left_index = True, right_index = True)
IndecesWdayPM = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,PTI_SL_PM,TTI_85th_PM,PTI_85th_PM,BTI_PM,TTI_85thWd_AM,PTI_85thWd_AM,TTI_85thWd_MD,PTI_85thWd_MD,TTI_85thWd_PM,PTI_85thWd_PM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,2.769,1.277,2.538,0.988,1.249,2.345,1.242,2.267,1.276,2.407


In [172]:
master = pd.merge(master, IndecesWdtimeAM, left_index = True, right_index = True)
IndecesWdtimeAM = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,TTI_85thWd_AM,PTI_85thWd_AM,TTI_85thWd_MD,PTI_85thWd_MD,TTI_85thWd_PM,PTI_85thWd_PM,TTI_60thWdtime_AM,PTI_60thWdtime_AM,TTI_85thWdtime_AM,PTI_85thWdtime_AM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,1.249,2.345,1.242,2.267,1.276,2.407,1.083,2.034,1.249,2.345


In [173]:
master = pd.merge(master, IndecesWdtimeMD, left_index = True, right_index = True)
IndecesWdtimeMD = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,TTI_85thWd_PM,PTI_85thWd_PM,TTI_60thWdtime_AM,PTI_60thWdtime_AM,TTI_85thWdtime_AM,PTI_85thWdtime_AM,TTI_60thWdtime_MD,PTI_60thWdtime_MD,TTI_85thWdtime_MD,PTI_85thWdtime_MD
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,1.276,2.407,1.083,2.034,1.249,2.345,1.078,1.967,1.242,2.267


In [174]:
master = pd.merge(master, IndecesWdtimePM, left_index = True, right_index = True)
IndecesWdtimePM = pd.DataFrame(columns = [])
master.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,...,TTI_85thWdtime_AM,PTI_85thWdtime_AM,TTI_60thWdtime_MD,PTI_60thWdtime_MD,TTI_85thWdtime_MD,PTI_85thWdtime_MD,TTI_60thWdtime_PM,PTI_60thWdtime_PM,TTI_85thWdtime_PM,PTI_85thWdtime_PM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
998969850F,42.87,18.639,474,0.692,8.84,42.87,18.018,2799,3.826,9.22,...,1.249,2.345,1.078,1.967,1.242,2.267,1.08,2.037,1.276,2.407


""


In [35]:
master.to_csv('HEREstats.csv')